<a href="https://colab.research.google.com/github/knobs-dials/wetsuite-dev/blob/main/notebooks/extras/datacollect/extras_datacollect_kamervragen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Purpose of this notebook

Take kamervragen text out of the data fetched by [extras_datacollect_frbr](extras_datacollect_frbr.ipynb) / _pod.

In [1]:
import re
import collections
import random
import pprint

import wetsuite.helpers.etree
import wetsuite.helpers.format
import wetsuite.helpers.localdata
import wetsuite.helpers.strings

In [3]:
frbr_fetched  = wetsuite.helpers.localdata.LocalKV('frbr_fetched.db', key_type=str, value_type=bytes )

### remind ourselves what's in there


In [4]:
random.sample( frbr_fetched.keys(), 20 )

['https://repository.overheid.nl/frbr/lokalebekendmakingen/d9fc747f449da6d47c17dc68f237056a/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/gmb/2018/gmb-2018-158898/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/gmb/2020/gmb-2020-140070/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20192020/ah-tk-20192020-1576/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/lokalebekendmakingen/895fa72caf95e3d1e97b9f4ae9fe638a/1/html/gmb-2024-11830.html',
 'https://repository.overheid.nl/frbr/officielepublicaties/gmb/2016/gmb-2016-67638/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/gmb/2014/gmb-2014-13017/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/sgd/19671968/0000238120/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/sgd/19721973/0000211412/1/metadata/didl.xml',
 'https://repository.overheid.n

In [7]:
# okay, lots of different things, can we quickly get a count of what seem to be the the collections?
count_collections = collections.defaultdict(int)

for url in frbr_fetched.keys():
    if url.startswith( 'https://repository.overheid.nl/frbr/'):
        sp = url.split('/')[4]
        count_collections[ sp ] += 1
    else:
        count_collections[ 'unsorted' ] += 1

dict( count_collections )

{'datacollecties': 22,
 'lokalebekendmakingen': 431956,
 'officielepublicaties': 2253406,
 'samenwerkendecatalogi': 55633,
 'sgd': 407218,
 'tuchtrecht': 38084,
 'vd': 1720}

In [8]:
# focus on just officiele publicaties - get the the document categories in there
count_optypes = collections.defaultdict(int)

# remind ourselves what's in there
for url in frbr_fetched.keys():
    if url.startswith( 'https://repository.overheid.nl/frbr/officielepublicaties/'):
        sp = url.split('/')[5]
        count_optypes[ sp ] += 1

dict( count_optypes )

{'ag-ek': 3231,
 'ag-tk': 769,
 'ag-vv': 104,
 'ag': 112,
 'ah-ek': 1600,
 'ah-tk': 315031,
 'ah': 187,
 'bgr': 3046,
 'blg': 12533,
 'gmb': 1097244,
 'h-ek': 37039,
 'h-tk': 83396,
 'h-vv': 204,
 'kst': 101720,
 'kv-ek': 388,
 'kv-tk': 142163,
 'kv': 7611,
 'nds-tk': 17039,
 'nds': 765,
 'prb': 175730,
 'stb': 11803,
 'stcrt': 197400,
 'trb': 4306,
 'wsb': 39985}

In [5]:
# it seems that we care only about ah-tk  (detailed elsewhere TODO: mention where)
# so collect those ang group that documents for each identifier into sets

# the URLs look something like   
#   'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20012002/ah-tk-20012002-1580/1/metadata/metadata.xml'
# where 
#   we ignore '20012002'
#   and consider 'ah-tk-20012002-1580' an identifier
#   we consider 'metadata' the type

ah_tk_sets = collections.defaultdict( list )

for url in frbr_fetched.keys():  #                                                              date  ident    typ 
    m = re.match('https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/[^/]+/([^/]+)/[^/]+/([^/]+)/', url)
    if m is not None:
        id, typ = m.groups()
        ah_tk_sets[id].append( (typ, url) )


print( "Amount of document sets: %s"%len(ah_tk_sets) )

# as an illustration of what we just made:
random.sample(  list(ah_tk_sets.items()), 2  )

Amount of document sets: 78929


[('ah-tk-20122013-2779',
  [('html',
    'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20122013/ah-tk-20122013-2779/1/html/ah-tk-20122013-2779.html'),
   ('metadata',
    'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20122013/ah-tk-20122013-2779/1/metadata/metadata.xml'),
   ('metadataowms',
    'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20122013/ah-tk-20122013-2779/1/metadataowms/metadata_owms.xml'),
   ('xml',
    'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20122013/ah-tk-20122013-2779/1/xml/ah-tk-20122013-2779.xml')]),
 ('ah-tk-20182019-1313',
  [('html',
    'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20182019/ah-tk-20182019-1313/1/html/ah-tk-20182019-1313.html'),
   ('metadata',
    'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20182019/ah-tk-20182019-1313/1/metadata/metadata.xml'),
   ('metadataowms',
    'https://repository.overheid.nl/frbr/officielepublicaties/ah-t

In [6]:
# define some helpers functions that we're about to use
def first_of_type(set, typ):
    " given a list-of-tuples as just mentioned, and e.g. 'xml', return the according URL - the first/any, if there happen to be multiple "
    for styp, surl in set:
        if typ==styp:
            return surl
    return None


def simpler_metadata(tree):
    ' eases reading metadata.xml '
    ret = collections.defaultdict(list) # returned as key -> list of values (often just one)
    for metadata in tree.findall('metadata'):
        #print( wetsuite.helpers.etree.tostring(metadata) )
        name = metadata.get('name')
        if name==None:
            continue
        name = name.split('.')[-1]
        ret[ name ].append( metadata.get('content') )
    return ret


def simpler_metadata_owms(tree):
    ' eases reading metadata_owms.xml '
    ret = collections.defaultdict(list) # returned as key -> list of values (often just one)
    tree = etree.strip_namespace(tree)

    for (path, key) in (
        ('owmskern/type',        'type'),
        ('owmsmantel/available', 'available'),
        ('owmsmantel/available', 'issued'),
        ('oep/available',        'aanhangselNummer'),
        ('oep/indiener',         'indiener'),
        ('oep/ontvanger',        'ontvanger'),
    ):
        tags = tree.findall(path)
        if len(tags)>0:
            ret[key] = list( tag.text   for tag in tags)
    for metadata in tree.findall('metadata'):
        ret[ metadata.get('name') ].append( metadata.get('content') )
    return ret

In [7]:
# most of the data going into the dataset later, which we are about to start putting in there
collected       = {}  # identifier -> detail_dict


count_postponed = 0

for id in ah_tk_sets:
    urls_in_set = ah_tk_sets[id]

    # start collecting the information going to the dataset
    item_merged = { 
        'urls':[]
    }

    ### Fetch metadata, and check whether it tells us there are answers in this set
    metadata_url      = first_of_type(urls_in_set, 'metadata')
    xml_url           = first_of_type(urls_in_set, 'xml')
    if metadata_url is None:
        #print( "SKIP, INCOMPLETE set %s, no metadata in %r"%(id, urls_in_set))
        continue
    if xml_url is None:
        #print( "SKIP, INCOMPLETE set %s, no xml in %r"%(id, urls_in_set))
        continue

    item_merged['urls'].append(metadata_url)
    item_merged['urls'].append(xml_url)
    
    metadata_bytestring = frbr_fetched.get( metadata_url )
    metadata_tree       = etree.fromstring( metadata_bytestring )
    metadata_dict       = simpler_metadata( metadata_tree )
    for key in metadata_dict:
        if key in ('issued', 'category', 'indiener', 'ontvanger', 'available','vergaderjaar', 'identifier', 'type'):
            v = metadata_dict[key]
            if key in ('available', 'issued', 'identifier' ,'category', 'vergaderjaar'):
                v = v[0]
            item_merged[key] = v

    if not 'Antwoord' in item_merged['type']:
        #print( "SKIP, no answers in document set %s   (%s)"%(id, merged['type']))
        continue

    ### There are answers, so actually fetch, parse, and collect
    xml_bytestring = frbr_fetched.get( xml_url )
    xml_tree       = etree.fromstring( xml_bytestring )
    xml_dict       = simpler_metadata( xml_tree )

    va = collections.defaultdict(dict) # number -> {details}

    # TODO/CONSIDER:
    # * what is kamervraagomschrijving?

    vragen     = xml_tree.findall('kamervragen/vraag')+xml_tree.findall('body/vragen/vraag')
    antwoorden = xml_tree.findall('kamervragen/antwoord')+xml_tree.findall('body/reactie/antwoord')
    if len(vragen)==0 or len(antwoorden)==0:
        # To see whether we are failing to parse any that are present, we filter out the "we will answer / did answer that later"
        #   and print what's left after that
        if strings.contains_any_of( xml_bytestring, [
            'gestelde termijn ','gebruikelijke termijn', 
            'zo spoedig mogelijk',
            'zijn beantwoord',
            'als ingetrokken worden beschouwd',
            ]):
            count_postponed += 1
        else:
            print( f'WARN: {len(vragen)} questions, {len(antwoorden)} answers in %r'%xml_url )

    for vraag in vragen:
        nr = vraag.find('nr')
        if nr is None or nr.text is None: # this looks like mild abuse of the format
            continue
        nr = re.sub('[^0-9]+', ' ', nr.text).strip() # removes the words and commas
        text = []
        for al in vraag.findall('al'):
            text.extend( etree.all_text_fragments( under_node=al, ignore_empty=True, ignore_tags=('extref','noot', 'noot.nr', 'noot.al') ) )
        va[ nr ]['vraag'] = ( ('\n\n'.join(text)).strip().replace('\xa0',' '),  etree.tostring(vraag, encoding='unicode') )


    for antwoord in antwoorden:
        nr = antwoord.find('nr')
        if nr is None or nr.text is None: # this looks like mild abuse of the format
            continue
        nr = re.sub('[^0-9]+', ' ', nr.text).strip()
        text = [] 
        for al in antwoord.findall('al'):
            text.extend( etree.all_text_fragments( under_node=al, ignore_empty=True, ignore_tags=('extref','noot', 'noot.nr', 'noot.al') ) )
        va[ nr ]['antwoord'] = ( ('\n\n'.join(text)).strip().replace('\xa0',' '),  etree.tostring(antwoord, encoding='unicode') )

    if len(va)==0:
        print("WARN: No useful question data in %r"%xml_url)
    #print( va )
    item_merged['vraagdata'] = dict(va) # defaultdict to dict

    collected[id] = item_merged   # TODO: finish

    # TODO: put amounts of document sets, and amount of questions, in description.
    #pprint.pprint( merged )

WARN: No useful question data in 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20022003/ah-tk-20022003-1391-h1/1/xml/ah-tk-20022003-1391-h1.xml'
WARN: No useful question data in 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20022003/ah-tk-20022003-486/1/xml/ah-tk-20022003-486.xml'
WARN: No useful question data in 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20072008/ah-tk-20072008-1362/1/xml/ah-tk-20072008-1362.xml'
WARN: No useful question data in 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20092010/ah-tk-20092010-1000/1/xml/ah-tk-20092010-1000.xml'
WARN: No useful question data in 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20092010/ah-tk-20092010-1001/1/xml/ah-tk-20092010-1001.xml'
WARN: No useful question data in 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20092010/ah-tk-20092010-1002/1/xml/ah-tk-20092010-1002.xml'
WARN: No useful question data in 'https://repository.o

In [10]:
print( "Number of document sets: %d\n"%len(collected) )

# print a few examples example
for id, detail_dict in random.sample( collected.items(), 3):
    print(id)
    pprint.pprint( detail_dict )

Number of document sets: 35299

ah-tk-20192020-2197
{'available': '2020-03-26',
 'category': 'Openbare orde en veiligheid | Criminaliteit',
 'identifier': 'ah-tk-20192020-2197',
 'indiener': ['W.L. Postma'],
 'issued': '2020-03-24',
 'ontvanger': ['F.B.J. Grapperhaus'],
 'type': ['officiële publicatie', 'Antwoord', 'Aanhangsel van de Handelingen'],
 'urls': ['https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20192020/ah-tk-20192020-2197/1/metadata/metadata.xml',
          'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20192020/ah-tk-20192020-2197/1/xml/ah-tk-20192020-2197.xml'],
 'vergaderjaar': '2019-2020',
 'vraagdata': {'1': {'antwoord': ('Ja.',
                                  '<antwoord '
                                  'xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n'
                                  '      <nr status="officieel">Antwoord '
                                  '1</nr>\n'
                                  '      <al>Ja.</al>

In [ ]:

kamervragen_dataset = localdata.MsgpackKV('kamervragen.db', key_type=str )     # URL -> dict

kamervragen_dataset._set_meta( 'description', '''
    Questions from members of the parliament (at the moment only tweede kamer) to the government.

    Source: The KOOP SRU repositories,  officielepublicaties set, and then only the ah-tk documents (aanhangsels, tweede kamer),
    and further restricted to the documents that contain answers (ignoring documents that are only the questions, or only the questions and a note that they will be answered later)

    The structure of .data looks something like:
       {'ah-tk-20102011-2857': {
            'identifier': 'ah-tk-20102011-2857',
            'urls':   [ 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20102011/ah-tk-20102011-2857/1/metadata/metadata.xml',
                        'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20102011/ah-tk-20102011-2857/1/xml/ah-tk-20102011-2857.xml',
                        ... 
                      ],
            'issued':       '2011-06-17',
            'available':    '2011-06-23',
            'vergaderjaar': '2010-2011',
            'category':     'Zorg en gezondheid | Jongeren',
            'type':         ['officiële publicatie', 'Antwoord', 'Aanhangsel van de Handelingen'],
            'indiener':     ['A.G. Wolbert'],
            'ontvanger':    ['M.L.L.E. Veldhuijzen van Zanten-Hyllner'],
            'vraagdata':    {
                '1': {
                      'vraag':    ['Bent u bekend met de tv-uitzending, waarin o.a. de moeder van Lucas haar verhaal doet?', '<vraag> Bent ...']
                   'antwoord':    ['Ja.',  '<antwoord> Ja. ...']
                },
                '2': {
                      'vraag':    ['Staat u nog steeds achter het principe dat mensen mogen kiezen uit zorg in een instelling en/of natura?', '<vraag> ...']]
                   'antwoord':    ['Mijn uitgangspunt blijft dat cliënten mogen kiezen bij welke zorgaanbieder zij de zorg in natura willen afnemen.', '<antwoord> ...']]
                },
       }}

    Note that 
    * the 'vraag' and 'antwoord' give a list of two items:
      * plain text, stripped of some references and notes, to have more natural text
      * the XML fragment it was taken from, in case you care about those references and notes

    * the keys in vraagdata are strings, not numbers directly, because...

    * questions and/or answers may be grouped, so you may e.g. see questions numbered '5' and '6' and an answer '5 6'.
      In this data structure, that means '5' and '6' will have only a 'vraag', and '5 6' will have only an 'antwoord'.
      This currently affects roughly 5% of questions.

    * The URLs mentioned are those that were actually used.  There are more, e.g. .pdf, .odt, html, and owms metadata


    TODO:
    * There are non-numbered questions in the XML, currently omitted - investigate
    * summarize how complete this data is with its source in general
    * there are mistakes in the XML we could fix (e.g. use of al instead of nr) 
                              

    ''') 

for id, detail_dict in collected.items():
       kamervragen_dataset.put( id, detail_dict )


